In [1]:
import torch
torch.cuda.is_available()
print(torch.zeros(1).cuda())

/home/asd27/.conda/envs/cudatorch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([0.], device='cuda:0')


In [2]:
import torch
import torch_geometric
import torch_geometric.nn as pyg_nn
from torch_sparse import SparseTensor, matmul
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
print(torch_geometric.__version__)

import torch_scatter
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.utils as pyg_utils

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType, OptTensor)

from torch.nn import Parameter, Linear
from torch_sparse import SparseTensor, set_diag
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax

import networkx as nx
from deepsnap.hetero_graph import HeteroGraph
import copy
from copy import deepcopy
import pickle

import deepsnap
from deepsnap.graph import Graph
from deepsnap.batch import Batch
from deepsnap.dataset import GraphDataset
from deepsnap.hetero_gnn import forward_op
from deepsnap.hetero_graph import HeteroGraph
from sklearn.metrics import f1_score, roc_auc_score

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import json
from pathlib import Path as Data_Path
from os import listdir
from os.path import isfile, join
from itertools import combinations
from tqdm.notebook import tqdm

2.1.0


In [3]:
import graph_tool.all as gt
print("graph-tool version: {}".format(gt.__version__.split(' ')[0]))

graph-tool version: 2.46


In [4]:
f = open('mpd.slice.0-999.json')
data = json.load(f)
print(type(data))

<class 'dict'>


In [22]:
def playlists2graph(g, data_json):
    #add playlist nodes
    playlist_attrs = []
    for playlist in data_json['playlists']:
        playlist_attrs.append({'name': playlist['name'], 'type': 'playlist'})
 
    g.vp['name'] = g.new_vertex_property('string')
    g.vp['type'] = g.new_vertex_property('string')
    for node_attrs in playlist_attrs:
        v = g.add_vertex()
        for key, value in node_attrs.items():
            g.vp[key][v] = value
            
def tracks2graph(g, tracks_list):
    #add track nodes
    track_attrs = []
    for track in tracks_list:
        track_attrs.append({'uri': track['track_uri'], 'name': track['track_name'], 'type': 'track'})

    g.vp['uri'] = g.new_vertex_property('string')
    g.vp['name'] = g.new_vertex_property('string')
    g.vp['type'] = g.new_vertex_property('string')
    for node_attrs in track_attrs:
        v = g.add_vertex()
        for key, value in node_attrs.items():
            g.vp[key][v] = value
        
def artists2graph(g, tracks_list):
    #add artist nodes
    artist_attrs = []
    for track in tracks_list:
        artist_attrs.append({'uri': track['artist_uri'], 'name': track['artist_name'], 'type': 'artist'})

    g.vp['uri'] = g.new_vertex_property('string')
    g.vp['name'] = g.new_vertex_property('string')
    g.vp['type'] = g.new_vertex_property('string')
    for i, node_attrs in enumerate(artist_attrs):
        v = g.add_vertex()
        for key, value in node_attrs.items():
            g.vp[key][v] = value

def append_graphs(g1, g2):
    # Get the last vertex index from g1
    last_vindex = g1.num_vertices() - 1

    # Get vertex properties from g1
    vp1 = {key: g1.vp[key] for key in g1.vertex_properties.keys()}

    # Add vertex properties to g2
    for key, value in vp1.items():
        g2.vp[key] = g2.new_vertex_property(value.value_type())

    # Append vertices from g1 to g2 with new index
    for v in g1.vertices():
        new_v = g2.add_vertex()
        for key, value in vp1.items():
            g2.vp[key][new_v] = value[v]
        

In [27]:
G = gt.Graph()
G.vp['uri'] = G.new_vertex_property('string')
G.vp['name'] = G.new_vertex_property('string')
G.vp['type'] = G.new_vertex_property('string')
# playlists2graph(g1, data)

# for i in range(len(data['playlists'])):
#     tracks2graph(g1, data['playlists'][i]['tracks'])
    

# for i in range(len(data['playlists'])):
g_temp = gt.Graph()
artists2graph(g_temp, data['playlists'][0]['tracks'])

append_graphs(g_temp, G)

In [28]:
G.num_vertices()

52

In [29]:
for v in G.vertices():
    print("Vertex", v, "Properties:")
    for prop in G.vertex_properties:
        print(prop, ":", G.vertex_properties[prop][v])

Vertex 0 Properties:
uri : spotify:artist:2wIVse2owClT7go1WT98tk
name : Missy Elliott
type : artist
Vertex 1 Properties:
uri : spotify:artist:26dSoYclwsYLMAKD3tpOr4
name : Britney Spears
type : artist
Vertex 2 Properties:
uri : spotify:artist:6vWDO969PvNqNYHIOW5v0m
name : Beyoncé
type : artist
Vertex 3 Properties:
uri : spotify:artist:31TPClRtHm23RisEBtV3X7
name : Justin Timberlake
type : artist
Vertex 4 Properties:
uri : spotify:artist:5EvFsr3kj42KNv97ZEnqij
name : Shaggy
type : artist
Vertex 5 Properties:
uri : spotify:artist:23zg3TcAtWQy7J6upgbUnj
name : Usher
type : artist
Vertex 6 Properties:
uri : spotify:artist:23zg3TcAtWQy7J6upgbUnj
name : Usher
type : artist
Vertex 7 Properties:
uri : spotify:artist:6wPhSqRtPu1UhRCDX5yaDJ
name : The Pussycat Dolls
type : artist
Vertex 8 Properties:
uri : spotify:artist:1Y8cdNmUJH7yBTd9yOvr5i
name : Destiny's Child
type : artist
Vertex 9 Properties:
uri : spotify:artist:1G9G7WwrXka3Z1r7aIDjI7
name : OutKast
type : artist
Vertex 10 Properties:
u

In [25]:
for v in G.vertices():
    print("Node", int(v), "has attrs:")
    for prop in G.vertex_properties:
        print("  -", prop, ":", G.vertex_properties[prop][v])

Node 0 has attrs:
  - uri : 
  - name : 
  - type : 
Node 1 has attrs:
  - uri : 
  - name : 
  - type : 
Node 2 has attrs:
  - uri : 
  - name : 
  - type : 
Node 3 has attrs:
  - uri : 
  - name : 
  - type : 
Node 4 has attrs:
  - uri : 
  - name : 
  - type : 
Node 5 has attrs:
  - uri : 
  - name : 
  - type : 
Node 6 has attrs:
  - uri : 
  - name : 
  - type : 
Node 7 has attrs:
  - uri : 
  - name : 
  - type : 
Node 8 has attrs:
  - uri : 
  - name : 
  - type : 
Node 9 has attrs:
  - uri : 
  - name : 
  - type : 
Node 10 has attrs:
  - uri : 
  - name : 
  - type : 
Node 11 has attrs:
  - uri : 
  - name : 
  - type : 
Node 12 has attrs:
  - uri : 
  - name : 
  - type : 
Node 13 has attrs:
  - uri : 
  - name : 
  - type : 
Node 14 has attrs:
  - uri : 
  - name : 
  - type : 
Node 15 has attrs:
  - uri : 
  - name : 
  - type : 
Node 16 has attrs:
  - uri : 
  - name : 
  - type : 
Node 17 has attrs:
  - uri : 
  - name : 
  - type : 
Node 18 has attrs:
  - uri : 
  - name

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
a,b,c = pl2graph(0, sample_playlist)

In [ ]:
a[0]

In [ ]:
a[1]

In [ ]:
a[2]

In [ ]:
G = nx.Graph()
G.add_nodes_from(a)

In [ ]:
G.nodes['pl_0']

In [ ]:
g = gt.Graph()

node_attrs = {'name': 'node1', 'color': 'red', 'size':10}
v = g.add_vertex()
g.vp['name'] = g.new_vertex_property('string')
g.vp['color'] = g.new_vertex_property('string')
g.vp['size'] = g.new_vertex_property('int')
for key, value in node_attrs.items():
    g.vp[key][v] = value